In [1]:
import urllib
import json
import pandas as pd
import numpy as np
import warnings
import pickle
from datetime import datetime
from datetime import timedelta
import statistics
warnings.filterwarnings('ignore')

In [2]:
#PICKLE_PATH = 'hopkins_conf_augmented0904.pkl'
PICKLE_PATH = 'hopkins_death_augmented0904.pkl'
#RESULT_PATH = 'hopkins.conf.0904.withgr.pkl'
RESULT_PATH = 'hopkins_death_withgr_augmented0904.pkl'


###### Open pickle file

In [6]:


hopkins_confirmed = pd.DataFrame()
 
hpcon_rates=pd.DataFrame()

with open(PICKLE_PATH, 'rb') as file:
    hopkins_confirmed = pickle.load(file)
    


hopkins_confirmed.insert(7,column='last relevant date',value=' ')

hopkins_confirmed.insert(7,column='first_7',value=np.nan)  # first 7 days after 5% inf croseed (if exist)
 
hopkins_confirmed.insert(7,column='EXP_GF_Q3',value=0.0 )     
hopkins_confirmed.insert(7,column='EXP_GF_Q2',value=0.0)  
hopkins_confirmed.insert(7,column='EXP_GF_Q1',value=0.0)   

hopkins_confirmed.insert(12,column='EXP_GF_ABOVE0',value=0.0 )
hopkins_confirmed.insert(12,column='EXP_GF_M01TO0',value=0.0 )
hopkins_confirmed.insert(13,column='EXP_GF_0TO01',value=0.0 )
hopkins_confirmed.insert(14,column='EXP_GF_01TO02',value=0.0 )
hopkins_confirmed.insert(15,column='EXP_GF_02TO03',value=0.0 )
hopkins_confirmed.insert(16,column='EXP_GF_03TO04',value=0.0 )
hopkins_confirmed.insert(17,column='EXP_GF_ABOVE04',value=0.0 )


print (hopkins_confirmed.columns)

print (hopkins_confirmed.columns[10:20])


hopkins_confirmed
    
        

#hpcon_datarows = pd.DataFrame()
#hpcon_datarows =hopkins_confirmed.index
    
#for row in hopkins_confirmed.iterrows():
#     hpcon_datarows.append(row)

#hpcon_datarows

Index(['Province_State', 'Country_Region', 'GDP', 'Urbanization', 'avg_m_tmp',
       'avg_m_RH', 'avg_m_precip', 'EXP_GF_Q1', 'EXP_GF_Q2', 'EXP_GF_Q3',
       ...
       '3/29/2020', '3/30/2020', '3/31/2020', '4/1/2020', '4/2/2020',
       '4/3/2020', '4/4/2020', '4/5/2020', '4/6/2020', '4/7/2020'],
      dtype='object', length=103)
Index(['first_7', 'last relevant date', 'EXP_GF_M01TO0', 'EXP_GF_0TO01',
       'EXP_GF_01TO02', 'EXP_GF_02TO03', 'EXP_GF_03TO04', 'EXP_GF_ABOVE04',
       'EXP_GF_ABOVE0', 'avg_m_wind'],
      dtype='object')


Province_State  \
coordinate                         information                                  
(-41.4545, 145.9707)               data                              Tasmania   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-40.9006, 174.886)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-38.4161, -63.6167)               data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-37.8136, 144.9631)               data                              Victoria   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.6751, -71.543)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.4735, 149.0124)               data          Australian Capital Territory   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
...                                                                       ...   
(61.14998174, -149.14269860000005) data                                Alaska   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(61.8926, -6.9118)                 data                         Faroe Islands   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(63.0, 16.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(64.0, 26.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
             

In [14]:
datarows = [row for row in hopkins_confirmed.iterrows()  if row[0][1]=='data']
#hopkins_confirmed.insert(7,column='last relevant date',value=np.nan)

 


### Obtain a list of dates (string of columns names which are dates , assumed to be a continouos sequence in the DF)


cur=0

for name in hopkins_confirmed.columns :
    if '2020' in name and '/' in name: # this is a date!
        datefound=True
        break
    cur +=1  
if datefound == False:
    print ('date not found in DF columns')
else:
    print ('first date is',name, 'offset',cur )
    firstdate=name
    firstdate_offset=cur 
    
rvrscur=len(hopkins_confirmed.columns)

for name in hopkins_confirmed.columns[::-1]:                #this loop will allow columns which are not dates right of dates
    if '2020' in name and '/' in name: # this is a date!
        break # this is  a date...
    else:
        rvrscur -=1
print ('rvrscur col',hopkins_confirmed.columns[rvrscur-1])
valid_dates=hopkins_confirmed.columns[cur:rvrscur]         # valid dates - all the dates in the index

print ('dates',valid_dates)
print ('last',valid_dates[-1])
 

for row in datarows:
    #print (row[0],row[0][1])
    ratios=[0]
    totals=[]
    deltas=[]
    last_tot=0
    actual_dates=[]
    
    for cursor in range (len(valid_dates)-1):
        p=row[1][valid_dates[cursor]]
        q=row[1][valid_dates[cursor+1]  ]
        actual_dates.append(valid_dates[cursor])
        
        
        
        if p > 0:
            ratios.append (round(q/p - 1,4 ))
            totals.append (p)
        else:
            if q>0:
                ratios.append (0)           # the first gr is set to 0  and will most often get removed due to
                                            # totals less then 5 pct,
            else: ratios.append(0)          #  0 growth between two zeros...
            totals.append(0)
            
    #ratios=[0]+ratios  # so that each growth rate 'sits' under its 'total' and is the growth between this and the 
                       # previous
    totals.append(row[1][valid_dates[-1]]) # add the last total to list
    actual_dates.append(valid_dates[-1])
   
    totals5=np.array(totals)
    
    mov_avg_5_totals= np.array([d/5 for d in totals5])
    ln=len(totals5)
    for j in range (1,5):
        mov_avg_5_totals[j:]=np.add(mov_avg_5_totals[j:], totals5[0:ln-j] /5 )
    
    deltas5=np.zeros_like(mov_avg_5_totals)
    for ind in range(1,len(deltas5)):
        deltas5[ind]=mov_avg_5_totals[ind]-mov_avg_5_totals[ind-1] 
    
    # calc the exponential growth (growth between this day mov_avg delta to prev. day mov_avg delta)
    mov_avg_5_ratio=np.zeros_like(deltas5)
    for ind in range (1,len(mov_avg_5_ratio)):
        if   deltas5[ind-1]==0 :  mov_avg_5_ratio[ind]=0
            
        else: mov_avg_5_ratio[ind]=   (deltas5[ind] / deltas5[ind-1]) - 1
            
    print ('==========')        
    print  ('new',row[1]['Province_State'],row[1][ 'Country_Region'])
    #print ('totals',totals)
    print ('totals5',totals5)
    print ('totals5 mov avg',mov_avg_5_totals)
    print ('deltas5',deltas5)
    print ('ratio5',mov_avg_5_ratio)
    e_ratios=mov_avg_5_ratio
       
    if max(totals)<20:
        print ('SKIP')
        continue    ###  if no day with 20 or more, skip this region,  small numbers 
    print ('NO SKIP')
        
   
    print (row[1]['Province_State'],row[1][ 'Country_Region'],ratios,totals)
    print ('lens',len(ratios),len(totals))
    maxt=max(totals)
    few=maxt/20
    print (row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    # here , find the 5% seperator
    few_seperator = -1
    first_confirmed_date   = None
    for cursor in range (len(totals)-1):
        
        if (first_confirmed_date == None) and totals[cursor] > 0 :   # foe exponential growth, we start from first positive
            first_confirmed_date=actual_dates[cursor]
            actual_exp_dates=actual_dates[cursor:]
            print ('exp cut, first confirmed, value, len before',first_confirmed_date,len(e_ratios))
            e_ratios=e_ratios[cursor:]
            e_sample=totals[cursor:]
            print ('exp cut, first confirmed, value, len after',first_confirmed_date,len(e_ratios))
          
            
        if  (totals[cursor] <=few ) and (totals [cursor+1]>=few):
            few_seperator=cursor
            break
    if few_seperator >=0 :
        t_sample=totals[cursor+1:]
        t_ratios=ratios[cursor+1:]
        actual_dates=actual_dates[cursor+1:]
        five_pct_name=actual_dates[0]
        
        print ('five pct', five_pct_name) 
    else:
        five_pct_name=actual_dates[0]  # this is the complete actual_dates
        print ('five pct not found',five_pct_name )
     # find the first positive confirmed date (for exponential date)
    
    #   find if there is a sequence of growth rate lower than 2% per day (from the end )  remove it if found
    
    for cur in range (len(t_ratios)):
        maxright=max(t_ratios[cur:])
        if   (maxright>0.02):
            continue
        else:
            print ('maxright ,cur, len ratios, rightside',maxright,cur,len(t_ratios),t_ratios[cur:],t_sample[cur:] )
            break
    
    
    last_relevant_sample=actual_dates[ -1]  # overriden if cut   
    if cur < len(t_ratios)-1:
     
        print ('cut right side len befor , len after',len(t_ratios),cur+1)
        
        t_ratios=t_ratios [:cur+1]
        t_sample=t_sample [:cur+1]
        actual_dates=actual_dates[:cur+1]
        last_relevant_sample=actual_dates[ -1]
        
        # here we cut also the exp relevant date, riding on previous calc of last relevant date
        print ('last date, e_ratio before',last_relevant_sample,e_ratios)
        for j in range(len(actual_exp_dates)):
            if actual_exp_dates[j]==last_relevant_sample: # cut the right side (same criteria as before, less than 2% )
                e_ratios=e_ratios[:j+1]
                e_sample=e_sample[:j+1]
                actual_exp_dates=actual_exp_dates[:j+1]
                break
        print ('last date, e_ratio after',last_relevant_sample,e_ratios)
            
            
        
    else: print ('nothing was cut')
            
    
    sorted_sample=t_sample.sort()
    
    
    print ('sample size',len(t_sample),t_sample,t_ratios)
    
    med=statistics.median(t_ratios)
    
    lower=np.percentile(t_ratios,25)
    upper=np.percentile(t_ratios,75)
    
    exp_med=round(statistics.median(e_ratios),4)
    
    exp_lower=round(np.percentile(e_ratios,25),4)
    exp_upper=round(np.percentile(e_ratios,75),4)
    print ('lower, median, upper, e..',lower,med,upper,exp_lower,exp_med,exp_upper)
    print ('e_ratios len, values',len(e_ratios),np.sort(e_ratios))
    
    npE=np.array(e_ratios)
    Epos=np.sum(npE > 0)
    EM01=((npE <= 0 ) & (npE>  -0.1)).sum()
    E1=((npE > 0 ) & (npE<= 0.1)).sum()
    E2=((npE > 0.1 ) & (npE<= 0.2)).sum()
    E3=((npE > 0.2 ) & (npE<= 0.3)).sum()
    E4=((npE > 0.3 ) & (npE<= 0.4)).sum()
    E5= (npE > 0.4 ).sum()
    
    print ('e POS m01 1 TO 5',EM01,Epos,E1,E2,E3,E4,E5)
    colslen=len(hopkins_confirmed)
    daycount=0
    sevendays=[]
    
    for smpl in t_sample:
        if smpl< few : continue
        else:
            daycount +=1
            if daycount==8 : break
            else: sevendays.append(smpl)
    if daycount==8:
        if sevendays[0]>0:
            
            seven_days_growth=(sevendays[-1]/sevendays[0])-1
    
        else:
            daycount=0
    else: daycount=0
     
    
    coord=row [0][0]
    if daycount>0:
        hopkins_confirmed.loc[coord, 'first_7']['data'] = seven_days_growth
        print ( 'Debug 7 ', seven_days_growth)
        
    hopkins_confirmed.loc[coord, '5%_Date']['data'] = five_pct_name
    hopkins_confirmed.loc[coord, 'last relevant date']['data']=last_relevant_sample
    hopkins_confirmed.loc[coord, 'GF_Q1']['data']=lower
    hopkins_confirmed.loc[coord, 'GF_Q2']['data']=med
    hopkins_confirmed.loc[coord, 'GF_Q3']['data']=upper
    hopkins_confirmed.loc[coord, 'EXP_GF_Q1']['data']=exp_lower
    hopkins_confirmed.loc[coord, 'EXP_GF_Q2']['data']=exp_med
    hopkins_confirmed.loc[coord, 'EXP_GF_Q3']['data']=exp_upper
    hopkins_confirmed.loc[coord, 'EXP_GF_ABOVE0']['data']=Epos
    
    #print ('Debug 0 ',hopkins_confirmed.loc[coord,'EXP_GF_Q1']['data'], 
     #      hopkins_confirmed.loc[coord,'EXP_GF_Q3']['data'])
  
    hopkins_confirmed.loc[coord,'EXP_GF_M01TO0']['data']=EM01 
    hopkins_confirmed.loc[coord,'EXP_GF_0TO01']['data']=E1 
    hopkins_confirmed.loc[coord,'EXP_GF_01TO02']['data']=E2
    hopkins_confirmed.loc[coord,'EXP_GF_02TO03']['data']=E3
    hopkins_confirmed.loc[coord,'EXP_GF_03TO04']['data']=E4
    hopkins_confirmed.loc[coord,'EXP_GF_ABOVE04']['data']=E5 
    #hopkins_confirmed.loc[(row[0][0],'Growth rates'),:]=   (  pd.Series([np.nan]*colslen) )     # add the row
    #print  ('row added ',row[0][0],row[0][1])
    
    #xx=hopkins_confirmed.loc[(row[0][0])].loc['data'].at['5%_Date'] 
        
    #hopkins_confirmed.loc[(row[0][0])].loc['data'].at['5%_Date'] =five_pct_name
    #xx2=hopkins_confirmed.loc[(row[0][0])].loc['data'].at['5%_Date'] 
    
    #print ('xx, xx2,xx3',xx,xx2 ) 
    #print ('last',last_relevant_sample)
    ##hopkins_confirmed.at[(row[0][0],'Growth rates'),'last relevant date']=last_relevant_sample
    #hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q1']=lower
    #hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q2']=med
    #hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q3']=upper
    #for cursor in range(len(actual_dates)):
    #    date=actual_dates[cursor]
    #    hopkins_confirmed.at[(row[0][0],'Growth rates'),date]=t_ratios[cursor]
    
    print ('DEBUG1' , row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    print ('DEBUG2' , lower, med, upper,five_pct_name,len(actual_dates),len(t_ratios))
    print ('DEBUG2E' , exp_lower, exp_med, exp_upper, len(actual_exp_dates),len(e_ratios))
    print ('DEBUG3', e_ratios)
    print ('DEBUG4', e_sample)
    print ('DEBUG5',hopkins_confirmed.loc[(row[0][0])])
    
    
      
        
hopkins_confirmed 
                 
 

first date is 1/22/2020 offset 26
rvrscur col 4/7/2020
dates Index(['1/22/2020', '1/23/2020', '1/24/2020', '1/25/2020', '1/26/2020',
       '1/27/2020', '1/28/2020', '1/29/2020', '1/30/2020', '1/31/2020',
       '2/1/2020', '2/2/2020', '2/3/2020', '2/4/2020', '2/5/2020', '2/6/2020',
       '2/7/2020', '2/8/2020', '2/9/2020', '2/10/2020', '2/11/2020',
       '2/12/2020', '2/13/2020', '2/14/2020', '2/15/2020', '2/16/2020',
       '2/17/2020', '2/18/2020', '2/19/2020', '2/20/2020', '2/21/2020',
       '2/22/2020', '2/23/2020', '2/24/2020', '2/25/2020', '2/26/2020',
       '2/27/2020', '2/28/2020', '2/29/2020', '3/1/2020', '3/2/2020',
       '3/3/2020', '3/4/2020', '3/5/2020', '3/6/2020', '3/7/2020', '3/8/2020',
       '3/9/2020', '3/10/2020', '3/11/2020', '3/12/2020', '3/13/2020',
       '3/14/2020', '3/15/2020', '3/16/2020', '3/17/2020', '3/18/2020',
       '3/19/2020', '3/20/2020', '3/21/2020', '3/22/2020', '3/23/2020',
       '3/24/2020', '3/25/2020', '3/26/2020', '3/27/2020', '3/28/20

DEBUG1 nan Argentina 56.0 2.8
DEBUG2 0.05095 0.14584999999999998 0.3333 3/19/2020 20 20
DEBUG2E -0.0 0.0 0.1652 31 31
DEBUG3 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16 -5.55111512e-16
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.11022302e-15
 -1.11022302e-15  0.00000000e+00 -1.00000000e+00  0.00000000e+00
  2.22044605e-15  1.00000000e+00  1.11022302e-15  0.00000000e+00
  5.00000000e-01  6.66666667e-01  8.88178420e-16  8.00000000e-01
  5.55555556e-01 -7.14285714e-02  1.53846154e-01  2.00000000e-01
 -1.66666667e-01  2.00000000e-01  1.11111111e-01 -8.88178420e-16
 -1.50000000e-01  1.76470588e-01 -1.77635684e-15]
DEBUG4 [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 4.0, 6.0, 8.0, 9.0, 13.0, 18.0, 19.0, 23.0, 27.0, 28.0, 36.0, 39.0, 43.0, 44.0, 48.0, 56.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN      Argentina

DEBUG1 nan Chile 43.0 2.15
DEBUG2 0.163325 0.22475 0.31479999999999997 3/25/2020 14 14
DEBUG2E 0.0 0.1905 0.3333 17 17
DEBUG3 [ 0.00000000e+00  1.00000000e+00 -4.44089210e-16  5.00000000e-01
  3.33333333e-01 -2.22044605e-16  6.66133815e-16  2.50000000e-01
  1.77635684e-15  6.00000000e-01  3.75000000e-01  9.09090909e-02
  2.50000000e-01  2.66666667e-01  1.57894737e-01 -4.54545455e-02
  1.90476190e-01]
DEBUG4 [1.0, 2.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 12.0, 16.0, 18.0, 22.0, 27.0, 34.0, 37.0, 43.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN          Chile  15923.35874        87.564   
avg_d_RH                NaN            NaN          NaN           NaN   
avg_d_precip            NaN            NaN          NaN           NaN   
avg_d_tmp               NaN            NaN          NaN           NaN   
avg_d_wind              NaN            NaN        

DEBUG1 New South Wales Australia 21.0 1.05
DEBUG2 0.0 0.0 0.1548 3/8/2020 31 31
DEBUG2E -0.0 0.0 0.1736 35 35
DEBUG3 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16 -5.55111512e-16
  1.00000000e+00 -5.00000000e-01  1.11022302e-15 -1.11022302e-15
  0.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.00000000e-01 -7.77156117e-16
  3.33333333e-01 -1.11022302e-15 -5.00000000e-01 -5.00000000e-01
  1.00000000e+00 -5.00000000e-01 -4.44089210e-15  4.44089210e-15
  1.00000000e+00 -5.00000000e-01  8.88178420e-15 -8.88178420e-15
  1.00000000e+00  4.44089210e-15  1.00000000e+00 -2.22044605e-15
  1.00000000e+00  1.25000000e-01  2.22222222e-01]
DEBUG4 [1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 4.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 8.0, 9.0, 10.0, 12.0, 12.0, 16.0, 18.0, 21.0]
DEBUG5                Province_State Country_Region          GDP  Urbanization  \
information                               

DEBUG1 nan Brazil 686.0 34.3
DEBUG2 0.18155 0.2163 0.2734 3/24/2020 15 15
DEBUG2E 0.1222 0.22 0.3456 22 22
DEBUG3 [ 0.          2.          1.          0.83333333  0.36363636  0.6
  0.29166667  0.29032258  0.2         0.29166667  0.08064516  0.14925373
  0.16883117  0.11111111  0.24        0.19354839  0.43918919  0.04694836
  0.28251121 -0.0034965   0.13684211  0.11728395]
DEBUG4 [1.0, 3.0, 6.0, 11.0, 15.0, 25.0, 34.0, 46.0, 59.0, 77.0, 92.0, 111.0, 136.0, 159.0, 201.0, 240.0, 324.0, 359.0, 445.0, 486.0, 564.0, 686.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN         Brazil  8920.762105        86.569   
avg_d_RH                NaN            NaN          NaN           NaN   
avg_d_precip            NaN            NaN          NaN           NaN   
avg_d_tmp               NaN            NaN          NaN           NaN   
avg_d_wind              NaN    

...

DEBUG1 nan United Kingdom 6159.0 307.95
DEBUG2 0.14652500000000002 0.21965 0.26242499999999996 3/23/2020 16 16
DEBUG2E 0.0738 0.199 0.3387 34 34
DEBUG3 [ 0.00000000e+00  1.00000000e+00 -4.44089210e-16  5.00000000e-01
  3.33333333e-01  2.50000000e-01  2.00000000e-01  4.44089210e-16
 -1.66666667e-01  2.40000000e+00 -1.17647059e-01  2.13333333e+00
 -3.33066907e-16  3.40425532e-01  8.41269841e-01  3.44827586e-01
  1.41025641e-01  2.69662921e-01  1.68141593e-01  7.95454545e-02
  1.05263158e-02  1.97916667e-01  3.85507246e-01  4.30962343e-01
  1.78362573e-01  1.69975186e-01  2.84199364e-01  3.15441784e-01
  1.93973635e-01  2.49737119e-01  2.22128734e-01  8.26161790e-02
 -3.94276630e-02  7.18305197e-02]
DEBUG4 [1.0, 2.0, 2.0, 3.0, 4.0, 6.0, 8.0, 8.0, 8.0, 21.0, 21.0, 55.0, 55.0, 71.0, 137.0, 177.0, 233.0, 281.0, 335.0, 422.0, 465.0, 578.0, 759.0, 1019.0, 1228.0, 1408.0, 1789.0, 2352.0, 2921.0, 3605.0, 4313.0, 4934.0, 5373.0, 6159.0]
DEBUG5              Province_State  Country_Region          

DEBUG1 nan Estonia 21.0 1.05
DEBUG2 0.0945 0.2019 0.31665 3/29/2020 10 10
DEBUG2E -0.0 0.05 0.3182 14 14
DEBUG3 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16 -5.55111512e-16
  2.00000000e+00 -3.33333333e-01  5.00000000e-01  3.33333333e-01
  1.50000000e+00 -1.00000000e-01  1.11111111e-01  1.00000000e-01
  2.72727273e-01 -2.85714286e-01]
DEBUG4 [1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 4.0, 5.0, 11.0, 12.0, 13.0, 15.0, 19.0, 21.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN        Estonia  23266.34627         68.88   
avg_d_RH                NaN            NaN          NaN           NaN   
avg_d_precip            NaN            NaN          NaN           NaN   
avg_d_tmp               NaN            NaN          NaN           NaN   
avg_d_wind              NaN            NaN          NaN           NaN   

              avg_m_tmp   avg_m_RH  avg_m_precip  EXP_GF

DEBUG1 nan Norway 89.0 4.45
DEBUG2 0.0704 0.1667 0.2105 3/18/2020 21 21
DEBUG2E 0.0 0.0 0.25 25 25
DEBUG3 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16  4.44089210e-16
  1.00000000e+00 -3.33333333e-01  6.66133815e-16  0.00000000e+00
  1.11022302e-15 -1.11022302e-15  2.50000000e-01  4.00000000e-01
  1.33226763e-15  7.14285714e-01  8.33333333e-02  6.66133815e-16
  3.84615385e-01  3.88888889e-01  1.33226763e-15  8.00000000e-02
  2.59259259e-01 -1.17647059e-01  6.66666667e-02 -2.22044605e-15
  2.18750000e-01]
DEBUG4 [3.0, 3.0, 3.0, 3.0, 6.0, 7.0, 7.0, 7.0, 7.0, 10.0, 12.0, 14.0, 14.0, 19.0, 23.0, 25.0, 32.0, 39.0, 44.0, 50.0, 59.0, 62.0, 71.0, 76.0, 89.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN         Norway  81697.24598        82.248   
avg_d_RH                NaN            NaN          NaN           NaN   
avg_d_precip            NaN           

DEBUG1 nan Sweden 591.0 29.55
DEBUG2 0.1187 0.239 0.32755 3/24/2020 15 15
DEBUG2E 0.0 0.1945 0.4803 28 28
DEBUG3 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16  1.00000000e+00
  5.00000000e-01  6.66666667e-01  2.00000000e-01  5.00000000e-01
  0.00000000e+00  4.44444444e-01  7.69230769e-02 -1.22124533e-15
  7.14285714e-02  6.66666667e-01  8.40000000e-01  2.39130435e-01
  4.73684211e-01 -4.76190476e-02 -7.50000000e-02  1.35135135e-01
  2.26190476e-01  3.00970874e-01  5.14925373e-01  2.21674877e-01
 -8.46774194e-02 -2.64317181e-02  7.69230769e-02  1.89075630e-01]
DEBUG4 [1.0, 1.0, 1.0, 2.0, 3.0, 6.0, 7.0, 10.0, 11.0, 16.0, 20.0, 21.0, 25.0, 36.0, 62.0, 77.0, 105.0, 105.0, 110.0, 146.0, 180.0, 239.0, 308.0, 358.0, 373.0, 401.0, 477.0, 591.0]
DEBUG5              Province_State Country_Region          GDP  Urbanization  \
information                                                             
data                    NaN         Sweden  54608.36025        87.431   
avg_d_RH                

Province_State  \
coordinate                         information                                  
(-41.4545, 145.9707)               data                              Tasmania   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-40.9006, 174.886)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-38.4161, -63.6167)               data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-37.8136, 144.9631)               data                              Victoria   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.6751, -71.543)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.4735, 149.0124)               data          Australian Capital Territory   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
...                                                                       ...   
(61.14998174, -149.14269860000005) data                                Alaska   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(61.8926, -6.9118)                 data                         Faroe Islands   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(63.0, 16.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(64.0, 26.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
             

In [10]:
look=hopkins_confirmed[[ 'Province_State',  'Country_Region','first_7']]
for row in look.iterrows() :
    if row[0][1]=='data' : 
         print (row[1][0:3])

Province_State     Tasmania
Country_Region    Australia
first_7                 NaN
Name: ((-41.4545, 145.9707), data), dtype: object
Province_State            NaN
Country_Region    New Zealand
first_7                   NaN
Name: ((-40.9006, 174.886), data), dtype: object
Province_State          NaN
Country_Region    Argentina
first_7             1.66667
Name: ((-38.4161, -63.6167), data), dtype: object
Province_State     Victoria
Country_Region    Australia
first_7                 NaN
Name: ((-37.8136, 144.9631), data), dtype: object
Province_State      NaN
Country_Region    Chile
first_7               3
Name: ((-35.6751, -71.543), data), dtype: object
Province_State    Australian Capital Territory
Country_Region                       Australia
first_7                                    NaN
Name: ((-35.4735, 149.0124), data), dtype: object
Province_State    South Australia
Country_Region          Australia
first_7                       NaN
Name: ((-34.9285, 138.6007), data), dtype: ob

Province_State    Oklahoma
Country_Region          US
first_7                NaN
Name: ((36.07695283, -96.97584944), data), dtype: object
Province_State    North Carolina
Country_Region                US
first_7                      NaN
Name: ((36.07959451, -79.78824948), data), dtype: object
Province_State    Oklahoma
Country_Region          US
first_7                NaN
Name: ((36.11939621, -95.94013939), data), dtype: object
Province_State    North Carolina
Country_Region                US
first_7                      NaN
Name: ((36.12859861, -80.25459052), data), dtype: object
Province_State    Tennessee
Country_Region           US
first_7                 NaN
Name: ((36.14005459, -85.50646123), data), dtype: object
Province_State         Gibraltar
Country_Region    United Kingdom
first_7                      NaN
Name: ((36.1408, -5.3536), data), dtype: object
Province_State    Tennessee
Country_Region           US
first_7                 NaN
Name: ((36.14669837, -87.35891709), data

Name: ((40.82925852, -81.88844833), data), dtype: object
Province_State    Pennsylvania
Country_Region              US
first_7                    NaN
Name: ((40.84785433, -76.70798222), data), dtype: object
Province_State    Nebraska
Country_Region          US
first_7                NaN
Name: ((40.85486962, -99.07500296), data), dtype: object
Province_State    New Jersey
Country_Region            US
first_7                  NaN
Name: ((40.85951833, -74.99556527), data), dtype: object
Province_State    New Jersey
Country_Region            US
first_7                5.875
Name: ((40.86095313, -74.54553709999998), data), dtype: object
Province_State    Utah
Country_Region      US
first_7            NaN
Name: ((40.86688146, -110.9535882), data), dtype: object
Province_State    Nebraska
Country_Region          US
first_7                NaN
Name: ((40.87257079, -98.50218293), data), dtype: object
Province_State    New York
Country_Region          US
first_7            1.35294
Name: ((40.88320

Name: ((53.1355, -57.6604), data), dtype: object
Province_State        NaN
Country_Region    Ireland
first_7           3.47368
Name: ((53.1424, -7.6921), data), dtype: object
Province_State        NaN
Country_Region    Belarus
first_7               NaN
Name: ((53.7098, 27.9534), data), dtype: object
Province_State    Manitoba
Country_Region      Canada
first_7                NaN
Name: ((53.7609, -98.8139), data), dtype: object
Province_State    Alberta
Country_Region     Canada
first_7                 3
Name: ((53.9333, -116.5765), data), dtype: object
Province_State       Isle of Man
Country_Region    United Kingdom
first_7                      NaN
Name: ((54.2361, -4.5481), data), dtype: object
Province_State          NaN
Country_Region    Lithuania
first_7                 NaN
Name: ((55.1694, 23.8813), data), dtype: object
Province_State               NaN
Country_Region    United Kingdom
first_7                  2.66567
Name: ((55.3781, -3.4360000000000004), data), dtype: object
Pro

In [13]:
hopkins_confirmed.to_csv('hopdeath.csv')

###### Calculate growth factors

In [11]:
hopkins_confirmed 
#hopkins_confirmed.columns



#dataonly=hopkins_confirmed['information']=='data'
#hopconf_data=hopkins_confirmed[dataonly]
#hopconf_data

Province_State  \
coordinate                         information                                  
(-41.4545, 145.9707)               data                              Tasmania   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-40.9006, 174.886)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-38.4161, -63.6167)               data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-37.8136, 144.9631)               data                              Victoria   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.6751, -71.543)                data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(-35.4735, 149.0124)               data          Australian Capital Territory   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
...                                                                       ...   
(61.14998174, -149.14269860000005) data                                Alaska   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(61.8926, -6.9118)                 data                         Faroe Islands   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(63.0, 16.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
                                   avg_d_precip                           NaN   
                                   avg_d_tmp                              NaN   
                                   avg_d_wind                             NaN   
(64.0, 26.0)                       data                                   NaN   
                                   avg_d_RH                               NaN   
             

###### Save file as pickle

In [12]:
pickle_path = RESULT_PATH.format(datetime.now().strftime('%d%m'))
with open(pickle_path, 'wb') as file:
    pickle.dump(hopkins_confirmed, file)